# Openbb-agents 智能体代理
这是一个简短的示例笔记本，用来展示`openbb-agents`的一些特性，并尽可能快地帮助你上手使用。

## Set-up
We'll be fetching our data provider API keys using OpenBB Platform's backend for this example (it's free to sign up!).

You are also free to set-up your [API keys](https://docs.openbb.co/sdk/usage/api-keys) locally, or to rely only on `yfinance`
as a provider for data provider (although keep in mind that certain functions
will not be available as a result).

In [1]:
import os
OPENBB_PAT = os.environ.get('OPENBB_PAT')

## Basic Querying

In [2]:
from openbb_agents.agent import openbb_agent

result = openbb_agent("What is the current stock price of TSLA?", openbb_pat=OPENBB_PAT)
print(result)

2024-08-27 18:20:46,979 - INFO - openbb_agents.agent - Using all available OpenBB tools with providers: ['yfinance']
2024-08-27 18:20:46,981 - INFO - openbb_agents.tools - Building OpenBB tool vector index...


In [8]:
# You can turn off verbose output to only receive the answer.
result = openbb_agent("What is the current stock price of AAPL?", verbose=False)
print(result)

2024-08-27 17:55:13,124 - ERROR - ai21.http_client.http_client - Calling POST https://api.ai21.com/studio/v1 failed with a non-200 response code: 422 headers: Headers({'date': 'Tue, 27 Aug 2024 09:55:12 GMT', 'content-type': 'application/json', 'content-length': '52', 'connection': 'keep-alive', 'request-id': '4f672f5f-cda4-24ad-79eb-7e8dbe5780c3', 'via': '1.1 google', 'cf-cache-status': 'DYNAMIC', 'report-to': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v4?s=vJRSY2b%2FRwbpo6yBlO4AZWECdUoLQy8LFFFPJzqf5fUo5hys%2Bd%2BDA1mUCeGV9OeBHxT0xfmkpUkoXUnyTIvPD00Vb7hubqZNd%2F3TqQEiCeMV1qca5uE8xoM2TRJvSOMN%2F2BwEhuBO3Tk%2BA%3D%3D"}],"group":"cf-nel","max_age":604800}', 'nel': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'strict-transport-security': 'max-age=15552000; includeSubDomains', 'server': 'cloudflare', 'cf-ray': '8b9b2f39c802ba30-SEA'})


UnprocessableEntity: Failed with http status code: 422 (UnprocessableEntity). Details: {"detail":"Each text must be under 2000 characters"}

## Use only specific OpenBB Platform tools
You don't have to rely on the entire suite of OpenBB tools. In certain
use-cases, it makes sense to limit the agent to only a small subset of the
tools.

In [9]:
# For example, if we only wanted our agent to use the `.equity.fundamental.income` tool:
from openbb import obb
obb.equity.fundamental.income("MSFT").to_df()

OpenBBError: 
[Error] -> Results not found.

In [10]:
#... then we can specify it using the `openbb_tools` input argument
result = openbb_agent("What is the most recent annual revenue of MSFT?", openbb_tools=[".equity.fundamental.income"])
print(result)

2024-08-27 17:55:51,981 - INFO - openbb_agents.agent - Using specified OpenBB tools: ['.equity.fundamental.income']
2024-08-27 17:55:54,427 - ERROR - ai21.http_client.http_client - Calling POST https://api.ai21.com/studio/v1 failed with a non-200 response code: 422 headers: Headers({'date': 'Tue, 27 Aug 2024 09:55:53 GMT', 'content-type': 'application/json', 'content-length': '52', 'connection': 'keep-alive', 'request-id': 'f0b3bd9a-5240-1979-9355-5065ae01a32f', 'via': '1.1 google', 'cf-cache-status': 'DYNAMIC', 'report-to': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v4?s=dTCAWvPj7UBYTBWJEnWark48Ya2yDheJ%2BtxJfhFxdMMCBB5bxJ41UgaxsnwDD%2BJYy7pG6Fbtwk2742KMmqLOnGVDNgv7SE9dmN1KJ3%2FVVXfoJ43XQyHaJoy49V7JvCvGpswdOGt7w%2B8Z1w%3D%3D"}],"group":"cf-nel","max_age":604800}', 'nel': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'strict-transport-security': 'max-age=15552000; includeSubDomains', 'server': 'cloudflare', 'cf-ray': '8b9b3041dcf17571-SEA'})


UnprocessableEntity: Failed with http status code: 422 (UnprocessableEntity). Details: {"detail":"Each text must be under 2000 characters"}

## Using custom tools
You're also not limited to OpenBB's suite of tools. If you have custom tools you'd like to specify and add to the tool index (so that the LLM can search for your tool while c ompleting tasks), you can do so simply by adding a custom tool to the agent.


In [6]:
import random

def get_weather(location: str) -> float:
    """Get the weather for a given location, in degress Celsius."""
    return random.uniform(0, 50)

# We can make extra tools available to the agent using the `extra_tools` input argument
result = openbb_agent("What is the weather at AMZN's headquarters?", extra_tools=[get_weather])
print(result)

2024-06-03 10:48:55,133 - INFO - openbb_agents.agent - Using all available OpenBB tools with providers: ['yfinance', 'fmp']
2024-06-03 10:48:55,134 - INFO - openbb_agents.tools - Building OpenBB tool vector index...
2024-06-03 10:49:08,001 - INFO - openbb_agents.tools - Adding user-specified tools to vector index...
2024-06-03 10:49:08,332 - INFO - openbb_agents.agent - Generating subquestions for user query: What is the weather at AMZN's headquarters?
2024-06-03 10:49:09,879 - INFO - openbb_agents.agent - Generated subquestions: [SubQuestion(id=1, question="Where is AMZN's headquarters located?", depends_on=None), SubQuestion(id=2, question="What is the current weather at the location of AMZN's headquarters?", depends_on=[1])]
2024-06-03 10:49:09,880 - INFO - openbb_agents.agent - Answering subquestion: id=1 question="Where is AMZN's headquarters located?" depends_on=None
2024-06-03 10:49:09,880 - INFO - openbb_agents.agent - Attempting to select tools for: {"Where is AMZN's headquart

## Async agent
An `async` agent is also available for use in your asnychronous workflows. This can lead to speed-ups in certain situations (we're still improving this over time!).

In [7]:
from openbb_agents.agent import aopenbb_agent

await aopenbb_agent("What is the stock price of AAPL and MSFT?")

2024-06-03 10:49:38,971 - INFO - openbb_agents.agent - Using all available OpenBB tools with providers: ['yfinance', 'fmp']
2024-06-03 10:49:38,972 - INFO - openbb_agents.tools - Building OpenBB tool vector index...
2024-06-03 10:49:53,849 - INFO - openbb_agents.agent - Pending subquestions: [SubQuestion(id=1, question='What is the current stock price of AAPL?', depends_on=[]), SubQuestion(id=2, question='What is the current stock price of MSFT?', depends_on=[])]
2024-06-03 10:49:53,850 - INFO - openbb_agents.agent - Answerable subquestions: [SubQuestion(id=1, question='What is the current stock price of AAPL?', depends_on=[]), SubQuestion(id=2, question='What is the current stock price of MSFT?', depends_on=[])]
2024-06-03 10:49:53,850 - INFO - openbb_agents.agent - Scheduling subquestion for answer: id=1 question='What is the current stock price of AAPL?' depends_on=[]
2024-06-03 10:49:53,851 - INFO - openbb_agents.agent - Scheduling subquestion for answer: id=2 question='What is the

'- The current stock price of Apple Inc. (AAPL) is $192.25.\n- The current stock price of Microsoft Corporation (MSFT) is $415.13.'